# PhageHostLearn - Klebsiella - inference

This is the notebook containing the code to run inference with a *Klebsiella* PhageHostLearn model, predicting interactions between phages and host in two folders of phage genome files and bacterial genome files.

### 1. Load libraries and set paths

In [1]:
import phl_utils as phlu
import pandas as pd
from tqdm.notebook import tqdm

path = '/Users/dimi/GoogleDrive/Phabric/2_DATA'

### 2. Process genomes

### 3. Compute representations

In [4]:
rbp_file = 'rbp_proteins.csv'
loci_file = 'loci_proteins.csv'

In [3]:
phlu.compute_representations(loci_file, path, suffix='_loci')

100%|██████████| 21356/21356 [8:15:45<00:00,  1.39s/it]   


In [ ]:
phlu.compute_representations(rbp_file, path, suffix='_rbps')

### 4. Run inference

In [2]:
# combine to multi-representations
filerbps = 'representations_rbps.csv'
phlu.combine_representations(filerbps, path, suffix='_rbps', mode='mean')

fileloci = 'representations_loci.csv'
phlu.combine_representations(fileloci, path, suffix='_loci', mode='mean')

100%|██████████| 16558/16558 [1:32:05<00:00,  3.00it/s]  


In [ ]:
### todel
import numpy as np
ints = pd.read_csv(path+'/interactions_rbplevel.csv')
new_ints = {'host': [], 'phage': [], 'label': []}
# for each unique host and phage in the interactions, check whether their unique interaction is in their, otherwise add with label = NaN
for host in tqdm(list(set(ints['host']))):
    for phage in list(set(ints['phage'])):
        if len(ints[(ints['host']==host) & (ints['phage']==phage)]) == 0:
            new_ints['host'].append(host)
            new_ints['phage'].append(phage)
            new_ints['label'].append(np.nan)
ints = pd.concat([ints, pd.DataFrame(new_ints)], axis=0)
ints.reset_index(drop=True, inplace=True)
print('True?', len(ints) == len(set(ints['host']))*len(set(ints['phage'])))

In [2]:
# construct input features frame
filerbps = 'representations_rbps.csv'
fileloci = 'representations_loci.csv'
ints = pd.read_csv(path+'/interactions_rbplevel.csv')
features, interactions = phlu.construct_inputframe('multi_'+filerbps, 'multi_'+fileloci, path, interactions=ints)

100%|██████████| 12695/12695 [15:50<00:00, 13.36it/s]


In [ ]:
# make predictions
#model_file = 'phagehostlearn_esm2_xgb.json'
#predictions = phlu.predict_interactions(features, interactions, model_file, path, suffix='', mode='scores')

In [ ]:
# add the labels that are known to the preds, rest is NaN
# for those that are known, compute the recall
# for all (known and unknown), plot the score distribution (histogram)